In [172]:
import numpy 
import sklearn
import pandas as pd


In [201]:
Data_full = pd.read_csv("train.csv", index_col = "PassengerId")
features = ["Age", "CryoSleep", "VIP", 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', "HomePlanet","Destination", "Cabin"]

y = Data_full.Transported
X = Data_full.drop("Transported", axis=1)[features]

X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X, y)


# print(X_train[X_train["Cabin"].isna()])


cleaning

In [203]:
# ship side
X_train["portside"] = X_train["Cabin"].str[-1]
X_train["portside"] = X_train["portside"].map({"p":1, "S":0, })
X_train['portside'] = X_train["portside"].fillna(2)

X_valid["portside"] = X_valid["Cabin"].str[-1]
X_valid["portside"] = X_valid["portside"].map({"p":1, "S":0})
X_valid["portside"] = X_valid["portside"].fillna(2)


# # deck 
# X_train["portside"] = X_train["Cabin"].str[1]
# X_train["portside"] = X_train["portside"].map({"p":True, "S":False})

# X_valid["portside"] = X_valid["Cabin"].str[1]
# X_valid["portside"] = X_valid["portside"].map({"p":True, "S":False})

X_train.drop("Cabin", inplace=True, axis=1)
X_valid.drop("Cabin", inplace=True, axis=1)



In [204]:
from sklearn.impute import SimpleImputer

imputer_mode = SimpleImputer(strategy='most_frequent')
imputer_costant = SimpleImputer(strategy='constant', fill_value=0)
imputer_mean = imputer_spending = SimpleImputer(strategy='mean')


# fill na with 0
fill_NA_with_0 = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"] 
X_train[fill_NA_with_0] = imputer_costant.fit_transform(X_train[fill_NA_with_0])
X_valid[fill_NA_with_0] = imputer_costant.transform(X_valid[fill_NA_with_0])

# fill mode: boolena values 
bool_inputer_cols = ["CryoSleep", "VIP", "Destination", "HomePlanet"]
X_train[bool_inputer_cols] = imputer_mode.fit_transform(X_train[bool_inputer_cols])
X_valid[bool_inputer_cols] = imputer_mode.transform(X_valid[bool_inputer_cols])

# fill adv: age
adv_inputer_cols = ["Age"]
X_train[adv_inputer_cols] = imputer_mean.fit_transform(X_train[adv_inputer_cols])
X_valid[adv_inputer_cols] = imputer_mean.transform(X_valid[adv_inputer_cols])


# for col in X_train.columns:
    # print(X_train[col].value_counts())


# for col in X_train.columns:
#     print(col, X_train[col].isna().sum())

# print()
# for i in Data_full.columns:
#     print(i)




In [205]:
# one hot encoding
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()


# # one hot encode Home Planet
train_encoded = encoder.fit_transform(X_train[['HomePlanet']])
encoded_df_train = pd.DataFrame(train_encoded.toarray(), columns=encoder.get_feature_names_out(['HomePlanet']))
encoded_df_train.index = X_train.index
X_train = pd.concat([X_train, encoded_df_train], axis=1)
X_train.drop("HomePlanet", axis=1, inplace=True)

val_encoded = encoder.transform(X_valid[['HomePlanet']])                 # ask chat how to put keep column names
encoded_df_val = pd.DataFrame(val_encoded.toarray(), columns=encoder.get_feature_names_out(['HomePlanet']))
encoded_df_val.index = X_valid.index
X_valid = pd.concat([X_valid, encoded_df_val], axis=1)
X_valid.drop("HomePlanet", axis=1, inplace=True)


# one hot encode destination
train_encoded = encoder.fit_transform(X_train[['Destination']])
encoded_df_train = pd.DataFrame(train_encoded.toarray(), columns=encoder.get_feature_names_out(['Destination']))
encoded_df_train.index = X_train.index
X_train = pd.concat([X_train, encoded_df_train], axis=1)
X_train.drop("Destination", axis=1, inplace=True)

val_encoded = encoder.transform(X_valid[['Destination']])                 # ask chat how to put keep column names
encoded_df_val = pd.DataFrame(val_encoded.toarray(), columns=encoder.get_feature_names_out(['Destination']))
encoded_df_val.index = X_valid.index
X_valid = pd.concat([X_valid, encoded_df_val], axis=1)
X_valid.drop("Destination", axis=1, inplace=True)


X_train


,Age,CryoSleep,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,portside,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
PassengerId,,,,,,,,,,,,,,,
7039_01,36.0,False,False,56.0,0.0,81.0,0.0,468.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0
6421_02,29.0,False,False,0.0,577.0,0.0,5568.0,432.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0
0639_01,63.0,False,False,0.0,123.0,956.0,25.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0
6263_01,17.0,True,False,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0
4720_01,21.0,False,False,0.0,681.0,51.0,131.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6599_01,49.0,True,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1967_01,30.0,True,False,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0
5264_02,37.0,False,False,0.0,0.0,163.0,0.0,486.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [206]:
# train model

model = sklearn.ensemble.RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)


preds = model.predict(X_valid)
print("MAE:")
print(sklearn.metrics.mean_absolute_error(y_valid, preds) * 100)

# runs 
# 1 : 31.4543
# 2: 28.408
# 3: 27.7449
# 4: 27.9839



MAE:
28.524165348330367
